### importação dos dados

In [3]:
import pandas as pd 
import numpy as np 
import sqlite3

### Leitura dos dados

In [4]:
financial_sample = pd.read_csv('financial_sample.csv')

In [5]:
financial_sample

,id,Segment,Country,Product,Discount_Band,Units_Sold,Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit,Date,Month_Number,Month Name,Year
0,0,Government,Canada,Carretera,None,1618.5,1618.5,20.0,32370.0,0.00,32370.00,16185.0,16185.00,1/1/2014,1,January,2014
1,1,Government,Germany,Carretera,None,1321.0,1321.0,20.0,26420.0,0.00,26420.00,13210.0,13210.00,1/1/2014,1,January,2014
2,2,Midmarket,France,Carretera,None,2178.0,2178.0,15.0,32670.0,0.00,32670.00,21780.0,10890.00,6/1/2014,6,June,2014
3,3,Midmarket,Germany,Carretera,None,888.0,888.0,15.0,13320.0,0.00,13320.00,8880.0,4440.00,6/1/2014,6,June,2014
4,4,Midmarket,Mexico,Carretera,None,2470.0,2470.0,15.0,37050.0,0.00,37050.00,24700.0,12350.00,6/1/2014,6,June,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,695,Small Business,France,Amarilla,High,2475.0,2475.0,300.0,742500.0,111375.00,631125.00,618750.0,12375.00,3/1/2014,3,March,2014
696,696,Small Business,Mexico,Amarilla,High,546.0,546.0,300.0,163800.0,24570.00,139230.00,136500.0,2730.00,10/1/2014,10,October,2014
697,697,Government,Mexico,Montana,High,1368.0,1368.0,7.0,9576.0,1436.40,8139.60,6840.0,1299.60,2/1/2014,2,February,2014
698,698,Government,Canada,Paseo,High,723.0,723.0,7.0,5061.0,759.15,4301.85,3615.0,686.85,4/1/2014,4,April,2014


### Divisão das colunas para modelo Star Schema - Dimensões 

*** Dim_Tempo

In [6]:
dim_tempo = pd.DataFrame({
    
    "data": financial_sample['Date'], 
})

In [7]:
### Verificação de dados nulos
dim_tempo.isna().sum()

data    0
dtype: int64

In [8]:
### Verificação de dados duplicados 
dim_tempo.duplicated().sum()

684

In [9]:
### Removendo dados duplicados 
dim_tempo = dim_tempo.drop_duplicates()
dim_tempo_count = dim_tempo.count()[0]

In [10]:
pd.DataFrame({"Total do tempo" : [dim_tempo_count]})

,Total do tempo
0,16


In [11]:
# Converter a coluna 'data' para datetime
dim_tempo['data'] = pd.to_datetime(dim_tempo['data'], format='%d/%m/%Y')

In [12]:
# Converter para o formato 'yyyy-m-d' sem zeros à esquerda
dim_tempo['data'] = dim_tempo['data'].dt.strftime('%Y-%-m-%-d')

In [13]:
dim_tempo

,data
0,2014-1-1
2,2014-1-6
5,2014-1-12
6,2014-1-3
11,2014-1-7
12,2014-1-8
13,2014-1-9
14,2013-1-10
16,2014-1-2
22,2013-1-9


*** Fim do Dim_Tempo 

### Dim_Produto 

In [14]:
dim_produto = pd.DataFrame({
    
    "produto": financial_sample['Product'], 
    "preco_de_fabricacao": financial_sample['Manufacturing_Price'], 
    "preco_de_venda": financial_sample['Sale_Price']
})

In [15]:
### Verificação de dados nulos
dim_produto.isna().sum()

produto                0
preco_de_fabricacao    0
preco_de_venda         0
dtype: int64

In [16]:
### Verificação de dados duplicados 
dim_produto.duplicated().sum()

5

In [17]:
### Removendo dados duplicados 
dim_produto = dim_produto.drop_duplicates()
dim_produto_count = dim_produto.count()[0]

In [18]:
pd.DataFrame({"Total dos produtos" : [dim_produto_count]})

,Total dos produtos
0,695


In [19]:
dim_produto

,produto,preco_de_fabricacao,preco_de_venda
0,Carretera,1618.5,20.0
1,Carretera,1321.0,20.0
2,Carretera,2178.0,15.0
3,Carretera,888.0,15.0
4,Carretera,2470.0,15.0
...,...,...,...
695,Amarilla,2475.0,300.0
696,Amarilla,546.0,300.0
697,Montana,1368.0,7.0
698,Paseo,723.0,7.0


### Dim_País 

In [20]:
dim_pais = pd.DataFrame({
    
    "pais": financial_sample['Country'],
})

In [21]:
### Verificação de dados nulos
dim_pais.isna().sum()

pais    0
dtype: int64

In [22]:
### Verificação de dados duplicados 
dim_pais.duplicated().sum()

695

In [23]:
### Removendo dados duplicados 
dim_pais = dim_pais.drop_duplicates()
dim_pais_count = dim_pais.count()[0]

In [24]:
pd.DataFrame({"Total dos pais" : [dim_pais_count]})

,Total dos pais
0,5


In [25]:
### Fim do Dim_Pais
dim_pais

,pais
0,Canada
1,Germany
2,France
4,Mexico
15,United States of America


### Dim Segmento 


In [26]:
dim_segmento = pd.DataFrame({
    
    "segmento": financial_sample['Segment']
})

In [27]:
### Verificação de dados nulos
dim_segmento.isna().sum()

segmento    0
dtype: int64

In [28]:
### Verificação de dados duplicados 
dim_segmento.duplicated().sum()

695

In [29]:
### Removendo dados duplicados 
dim_segmento = dim_segmento.drop_duplicates()
dim_segmento_count = dim_segmento.count()[0]

In [30]:
pd.DataFrame({"Total dos Segmentos" : [dim_segmento_count]})

,Total dos Segmentos
0,5


In [31]:
### Fim do Dim_Segmento
dim_segmento

,segmento
0,Government
2,Midmarket
7,Channel Partners
11,Enterprise
12,Small Business


### Dim Desconto 

In [32]:
dim_desconto = pd.DataFrame({
    
    "faixa_desconto": financial_sample['Discount_Band']
})

In [33]:
### Verificação de dados nulos
dim_desconto.isna().sum()

faixa_desconto    0
dtype: int64

In [34]:
### Verificação de dados duplicados 
dim_desconto.duplicated().sum()

696

In [35]:
### Removendo dados duplicados 
dim_desconto = dim_desconto.drop_duplicates()
dim_desconto_count = dim_desconto.count()[0]

In [36]:
pd.DataFrame({"Total dos descontos" : [dim_desconto_count]})

,Total dos descontos
0,4


In [37]:
dim_desconto

,faixa_desconto
0,None
53,Low
213,Medium
455,High


### Fatos Vendas 

In [38]:
fatos_vendas = pd.DataFrame({
    
    "unidades_vendidas": financial_sample['Units_Sold'], 
    "vendas_brutas": financial_sample['Gross_Sales'], 
    "descontos": financial_sample['Discounts'], 
    "vendas": financial_sample['Sales'], 
    "cogs": financial_sample['COGS'], 
    "lucro": financial_sample['Profit'], 
    "data": financial_sample['Date']
})

In [39]:
### Verificação de dados nulos
fatos_vendas.isna().sum()

unidades_vendidas    0
vendas_brutas        0
descontos            0
vendas               0
cogs                 0
lucro                0
data                 0
dtype: int64

In [40]:
### Verificação de dados duplicados 
fatos_vendas.duplicated().sum()

140

In [41]:
### Removendo dados duplicados 
fatos_vendas = fatos_vendas.drop_duplicates()
fatos_vendas_count = fatos_vendas.count()[0]

In [42]:
pd.DataFrame({"Total dos Fatos de Vendas" : [fatos_vendas_count]})

,Total dos Fatos de Vendas
0,560


In [43]:
# Converter a coluna 'data' para datetime
fatos_vendas['data'] = pd.to_datetime(fatos_vendas['data'], format='%d/%m/%Y')

In [44]:
# Converter para o formato 'yyyy-m-d' sem zeros à esquerda
fatos_vendas['data'] = fatos_vendas['data'].dt.strftime('%Y-%-m-%-d')

In [45]:
fatos_vendas

,unidades_vendidas,vendas_brutas,descontos,vendas,cogs,lucro,data
0,1618.5,32370.0,0.00,32370.00,16185.0,16185.00,2014-1-1
1,1321.0,26420.0,0.00,26420.00,13210.0,13210.00,2014-1-1
2,2178.0,32670.0,0.00,32670.00,21780.0,10890.00,2014-1-6
3,888.0,13320.0,0.00,13320.00,8880.0,4440.00,2014-1-6
4,2470.0,37050.0,0.00,37050.00,24700.0,12350.00,2014-1-6
...,...,...,...,...,...,...,...
693,552.0,69000.0,10350.00,58650.00,66240.0,-7590.00,2014-1-11
695,2475.0,742500.0,111375.00,631125.00,618750.0,12375.00,2014-1-3
697,1368.0,9576.0,1436.40,8139.60,6840.0,1299.60,2014-1-2
698,723.0,5061.0,759.15,4301.85,3615.0,686.85,2014-1-4


## Conexão para o banco de dados 

In [46]:
dbconn = sqlite3.connect('dio_financeiro.db')

In [47]:
print(dbconn)

### Criação da Tabela de Banco de dados 

#### Dim_Tempo 

In [48]:
dbconn.execute(
    
    '''
    CREATE TABLE dim_tempo(
        
        id INTEGER PRIMARY KEY,
        data date
    )
    '''
)

dim_tempo.to_sql("dim_tempo", dbconn, if_exists='append', index=False)

16

In [49]:
pd.read_sql_query("SELECT * FROM dim_tempo", con=dbconn)

,id,data
0,1,2014-1-1
1,2,2014-1-6
2,3,2014-1-12
3,4,2014-1-3
4,5,2014-1-7
5,6,2014-1-8
6,7,2014-1-9
7,8,2013-1-10
8,9,2014-1-2
9,10,2013-1-9


#### Dim_Produto 

In [50]:
dbconn.execute(
    
    '''
    CREATE TABLE dim_produto(
    
        id INTEGER PRIMARY KEY,
        "produto" TEXT,
        preco_de_fabricacao DOUBLE,
        preco_de_venda DOUBLE
    )
    '''
)

dim_produto.to_sql("dim_produto", dbconn, if_exists='append', index=False)

695

In [51]:
pd.read_sql_query("SELECT * FROM dim_produto", con=dbconn)

,id,produto,preco_de_fabricacao,preco_de_venda
0,1,Carretera,1618.5,20.0
1,2,Carretera,1321.0,20.0
2,3,Carretera,2178.0,15.0
3,4,Carretera,888.0,15.0
4,5,Carretera,2470.0,15.0
...,...,...,...,...
690,691,Amarilla,2475.0,300.0
691,692,Amarilla,546.0,300.0
692,693,Montana,1368.0,7.0
693,694,Paseo,723.0,7.0


#### Dim_Pais

In [52]:
dbconn.execute(
    
    '''
    CREATE TABLE dim_pais(
    
        id INTEGER PRIMARY KEY,
        "pais" TEXT
        
    )
    '''
)

dim_pais.to_sql("dim_pais", dbconn, if_exists='append', index=False)

5

In [53]:
pd.read_sql_query("SELECT * FROM dim_pais", con=dbconn)

,id,pais
0,1,Canada
1,2,Germany
2,3,France
3,4,Mexico
4,5,United States of America


#### Dim_Segmento 

In [54]:
dbconn.execute(
    
    '''
    CREATE TABLE dim_segmento(
    
        id INTEGER PRIMARY KEY,
        "segmento" TEXT
        
    )
    '''
)

dim_segmento.to_sql("dim_segmento", dbconn, if_exists='append', index=False)

5

In [55]:
pd.read_sql_query("SELECT * FROM dim_segmento", con=dbconn)

,id,segmento
0,1,Government
1,2,Midmarket
2,3,Channel Partners
3,4,Enterprise
4,5,Small Business


#### Dim_Desconto 

In [56]:
dbconn.execute(
    
    '''
    CREATE TABLE dim_desconto(
    
        id INTEGER PRIMARY KEY,
        "faixa_desconto" TEXT
        
    )
    '''
)

dim_desconto.to_sql("dim_desconto", dbconn, if_exists='append', index=False)

4

In [57]:
pd.read_sql_query("SELECT * FROM dim_desconto", con=dbconn)

,id,faixa_desconto
0,1,None
1,2,Low
2,3,Medium
3,4,High


#### Fatos_Vendas

In [58]:
dbconn.execute(
    
    '''
    CREATE TABLE fatos_vendas(
    
        id INTEGER PRIMARY KEY,
        unidades_vendidas DOUBLE, 
        vendas_brutas DOUBLE,
        descontos DOUBLE, 
        vendas DOUBLE, 
        cogs DOUBLE, 
        lucro DOUBLE, 
        data DATE
        
        
    )
    '''
)

fatos_vendas.to_sql("fatos_vendas", dbconn, if_exists='append', index=False)

560

In [59]:
pd.read_sql_query("SELECT * FROM fatos_vendas", con=dbconn)

,id,unidades_vendidas,vendas_brutas,descontos,vendas,cogs,lucro,data
0,1,1618.5,32370.0,0.00,32370.00,16185.0,16185.00,2014-1-1
1,2,1321.0,26420.0,0.00,26420.00,13210.0,13210.00,2014-1-1
2,3,2178.0,32670.0,0.00,32670.00,21780.0,10890.00,2014-1-6
3,4,888.0,13320.0,0.00,13320.00,8880.0,4440.00,2014-1-6
4,5,2470.0,37050.0,0.00,37050.00,24700.0,12350.00,2014-1-6
...,...,...,...,...,...,...,...,...
555,556,552.0,69000.0,10350.00,58650.00,66240.0,-7590.00,2014-1-11
556,557,2475.0,742500.0,111375.00,631125.00,618750.0,12375.00,2014-1-3
557,558,1368.0,9576.0,1436.40,8139.60,6840.0,1299.60,2014-1-2
558,559,723.0,5061.0,759.15,4301.85,3615.0,686.85,2014-1-4


### Análise dos dados da Planilha Financeiro Simples 

#### 1 - Calcular o total de vendas brutas ?? 

In [60]:
consulta_total_vendas_brutas = '''
    
    SELECT SUM(vendas_brutas) AS total_vendas_brutas FROM fatos_vendas
    
'''

In [134]:
resultado_1 = pd.read_sql_query(consulta_total_vendas_brutas, con=dbconn)

In [63]:
resultado_1["total_vendas_brutas"] = resultado_1["total_vendas_brutas"].map(lambda x: "R$ {:,.2f}".format(x))

In [64]:
resultado_1

,total_vendas_brutas
0,"R$ 104,971,922.50"


#### 2 - Calcular o total de vendas brutas: 

In [65]:
consulta_lucro_total = '''
    
    SELECT SUM(lucro) as lucro_total FROM fatos_vendas 
    
'''

In [66]:
resultado_2 = pd.read_sql_query(consulta_lucro_total, con=dbconn)

In [68]:
resultado_2["lucro_total"] = resultado_2["lucro_total"].map(lambda x: "R$ {:,.2f}".format(x))

In [69]:
resultado_2

,lucro_total
0,"R$ 13,424,051.92"


#### 3 - Vendas por país 

In [70]:
vendas_por_pais = '''
    
    SELECT dp.pais, SUM(fv.vendas) AS total_vendas 
    FROM fatos_vendas fv 
    JOIN dim_pais dp ON fv.id = dp.id 
    GROUP BY dp.pais
    
'''

In [71]:
resultado_3 = pd.read_sql_query(vendas_por_pais, con=dbconn)

In [73]:
resultado_3["total_vendas"] = resultado_3["total_vendas"].map(lambda x: "R$ {:,.2f}".format(x))

In [76]:
resultado_3 = resultado_3.sort_values(by='total_vendas', ascending=False)
resultado_3

,pais,total_vendas
4,United States of America,"R$ 37,050.00"
1,France,"R$ 32,670.00"
0,Canada,"R$ 32,370.00"
2,Germany,"R$ 26,420.00"
3,Mexico,"R$ 13,320.00"


#### 4 - Vendas por Segmento 

In [77]:
vendas_por_segmento = '''
    
    SELECT ds.segmento, SUM(fv.vendas) AS total_vendas 
    FROM fatos_vendas fv 
    JOIN dim_segmento ds ON fv.id = ds.id 
    GROUP BY ds.segmento 
    
'''

In [78]:
resultado_4 = pd.read_sql_query(vendas_por_segmento, con=dbconn)

In [80]:
resultado_4["total_vendas"] = resultado_4["total_vendas"].map(lambda x: "R$ {:,.2f}".format(x))

In [82]:
resultado_4 = resultado_4.sort_values(by='total_vendas', ascending=False)
resultado_4

,segmento,total_vendas
4,Small Business,"R$ 37,050.00"
0,Channel Partners,"R$ 32,670.00"
2,Government,"R$ 32,370.00"
3,Midmarket,"R$ 26,420.00"
1,Enterprise,"R$ 13,320.00"


#### 5 - Lucro por produto 

In [86]:
lucro_por_produto = '''
    
    SELECT dp.produto, SUM(fv.lucro) AS total_lucro 
    FROM fatos_vendas fv 
    JOIN dim_produto dp ON fv.id = dp.id 
    GROUP BY dp.produto 
    
'''

In [87]:
resultado_5 = pd.read_sql_query(lucro_por_produto, con=dbconn)

In [88]:
resultado_5["total_lucro"] = resultado_5["total_lucro"].map(lambda x: "R$ {:,.2f}".format(x))

In [89]:
resultado_5 = resultado_5.sort_values(by='total_lucro', ascending=False)
resultado_5

,produto,total_lucro
3,Paseo,"R$ 3,710,068.82"
5,Velo,"R$ 2,605,433.32"
2,Montana,"R$ 2,003,156.83"
1,Carretera,"R$ 1,967,132.90"
4,VTT,"R$ 1,753,510.76"
0,Amarilla,"R$ 1,384,749.29"


#### 6 - Análise COGS por Segmento

In [107]:
cogs_por_segmento = '''

    SELECT ds.segmento, SUM(fv.cogs) AS total_cogs 
    FROM fatos_vendas fv 
    JOIN dim_segmento ds ON fv.id = ds.id 
    GROUP BY ds.segmento
    
'''

In [108]:
resultado_6_1 = pd.read_sql_query(cogs_por_segmento, con=dbconn)

In [111]:
resultado_6_1["total_cogs"] = resultado_6_1["total_cogs"].map(lambda x: "R$ {:,.2f}".format(x)) 

In [117]:
resultado_6_1 = resultado_6_1.sort_values(by='total_cogs', ascending=False)

In [118]:
resultado_6_1

,segmento,total_cogs
1,Enterprise,"R$ 8,880.00"
4,Small Business,"R$ 24,700.00"
0,Channel Partners,"R$ 21,780.00"
2,Government,"R$ 16,185.00"
3,Midmarket,"R$ 13,210.00"


#### 7 - Análise COGS por País 

In [121]:
cogs_por_pais = '''
    
    SELECT dp.pais, SUM(fv.cogs) AS total_cogs 
    FROM fatos_vendas fv 
    JOIN dim_pais dp ON fv.id = dp.id 
    GROUP BY dp.pais
    
'''

In [122]:
resultado_7 = pd.read_sql_query(cogs_por_pais, con=dbconn)

In [124]:
resultado_7["total_cogs"] = resultado_7["total_cogs"].map(lambda x: "R$ {:,.2f}".format(x)) 

In [125]:
resultado_7 = resultado_7.sort_values(by='total_cogs', ascending=False)

In [126]:
resultado_7

,pais,total_cogs
3,Mexico,"R$ 8,880.00"
4,United States of America,"R$ 24,700.00"
1,France,"R$ 21,780.00"
0,Canada,"R$ 16,185.00"
2,Germany,"R$ 13,210.00"


#### 8 - Análise por COGS por Produto 

In [127]:
cogs_por_produto = '''
    
    SELECT dp.produto, SUM(fv.cogs) AS total_cogs 
    FROM fatos_vendas fv 
    JOIN dim_produto dp ON fv.id = dp.id 
    GROUP BY dp.produto
    
'''

In [128]:
resultado_8 = pd.read_sql_query(cogs_por_produto, con=dbconn)

In [129]:
resultado_8["total_cogs"] = resultado_8["total_cogs"].map(lambda x: "R$ {:,.2f}".format(x)) 

In [132]:
resultado_8 = resultado_8.sort_values(by='total_cogs', ascending=True)

In [133]:
resultado_8

,produto,total_cogs
4,VTT,"R$ 11,607,759.50"
2,Montana,"R$ 12,671,184.00"
1,Carretera,"R$ 13,535,576.50"
5,Velo,"R$ 18,006,980.50"
3,Paseo,"R$ 19,506,122.50"
0,Amarilla,"R$ 8,380,119.00"


#### 9 - Qual é o total e a média de preço de fabricação ?? 

In [149]:
precao_fabricacao_total = '''
    
    SELECT produto, SUM(preco_de_fabricacao) AS total, AVG(preco_de_fabricacao) AS media FROM dim_produto
    GROUP BY produto
    
'''

In [150]:
resultado_9 = pd.read_sql_query(precao_fabricacao_total, con=dbconn)

In [153]:
resultado_9["total"] = resultado_9["total"].map(lambda x: "R$ {:,.2f}".format(x)) 

In [154]:
resultado_9["media"] = resultado_9["media"].map(lambda x: "R$ {:,.2f}".format(x))

In [155]:
resultado_9 = resultado_9.sort_values(by='total', ascending=False)

In [156]:
resultado_9

,produto,total,media
3,Paseo,"R$ 334,615.50","R$ 1,681.48"
4,VTT,"R$ 168,783.00","R$ 1,548.47"
5,Velo,"R$ 161,761.50","R$ 1,497.79"
0,Amarilla,"R$ 155,315.00","R$ 1,652.29"
2,Montana,"R$ 151,856.00","R$ 1,650.61"
1,Carretera,"R$ 146,846.00","R$ 1,578.99"


#### 10 - Qual é o total e a média de preço de venda ?? 

In [157]:
precao_venda_total_media = '''
    
    SELECT produto, SUM(preco_de_venda) AS total, AVG(preco_de_venda) AS media FROM dim_produto
    GROUP BY produto
    
'''

In [158]:
resultado_10 = pd.read_sql_query(precao_venda_total_media, con=dbconn)

In [159]:
resultado_10["total"] = resultado_10["total"].map(lambda x: "R$ {:,.2f}".format(x)) 

In [160]:
resultado_10["media"] = resultado_10["media"].map(lambda x: "R$ {:,.2f}".format(x))

In [161]:
resultado_10 = resultado_10.sort_values(by='total', ascending=False)

In [162]:
resultado_10

,produto,total,media
3,Paseo,"R$ 21,810.00",R$ 109.60
4,VTT,"R$ 15,106.00",R$ 138.59
5,Velo,"R$ 12,436.00",R$ 115.15
0,Amarilla,"R$ 12,096.00",R$ 128.68
2,Montana,"R$ 10,878.00",R$ 118.24
1,Carretera,"R$ 10,395.00",R$ 111.77
